In [1]:
import numpy as np
import nltk
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
from sklearn.model_selection import train_test_split
from IPython import display

In [13]:
class Language_Model:
    def __init__(self, corpus):
        self.corpus = corpus
        self.max_ngram = 3
        self.Count_unigram = {}
        self.Count_bigram = {}
        self.Count_trigram = {}
        self.n_grams_counts()

    def n_grams_counts(self):
        Count = {}
        start = []
        end = ["</s>"]
        for i in range(max(1,self.max_ngram-1)):
            start.append("<s>")

        for tweet in self.corpus:
            words = start+tweet+end 
            for index, word in enumerate(words[self.max_ngram-1:]):
                context = [word]
                if word not in self.Count_unigram: 
                    self.Count_unigram[word] = 0
                self.Count_unigram[word]+=1

                for j in range(1,self.max_ngram): #Create n_gram
                    context.append(words[(index + self.max_ngram -1) - j])
                    if len(context) == 2:
                        if self.Count_bigram.get(str(context)) is not None: self.Count_bigram[str(context)] += 1
                        else: self.Count_bigram[str(context)] = 1
                    elif len(context) == 3:
                        if self.Count_trigram.get(str(context)) is not None: self.Count_trigram[str(context)] += 1
                        else: self.Count_trigram[str(context)] = 1
    
    def get_probabilities(context, word):
        pass

    def P_n_gram_sequence(self, sequence, n): #bigram,unigram,trigram
        words = tokenizer.tokenize(sequence)


In [3]:
def get_tweets_from_file(path_corpus):
    tr_tweet = []    
    with open(path_corpus, "r") as f_corpus:
        for tweet in f_corpus:
            tr_tweet += [tweet]
    return tr_tweet

In [4]:
def define_vocabulary(corpus, size):
    corpus_words = []
    for doc in corpus:
        corpus_words += tokenizer.tokenize(doc)
    fdist = nltk.FreqDist(corpus_words)
    V = [(fdist[key], key) for key in fdist]
    V.sort() 
    V.reverse()
    V = V[:size]
    V = [word for count, word in V]
    return V

In [5]:
def preprocess_corpus(corpus, volcabulary_size):
    # Make sure all tweets are in lowercase.
    for i in range(len(corpus)):
        corpus[i]= corpus[i].lower()
    # Define vocabulary with the 500 most frequent words.
    V = define_vocabulary(corpus, volcabulary_size)
    # Rebuild corpus
    new_corpus = []
    for tweet in corpus:
        words = tokenizer.tokenize(tweet)    
        new_words = ["<unk>" if word not in V else word for word in words] # New lexicon with unknown words
        new_corpus.append(new_words)
    return new_corpus, V

In [6]:
tr_tweet = get_tweets_from_file("./mex_train.txt")
tr_tweet, V = preprocess_corpus(tr_tweet, 500)

In [15]:
model = Language_Model(tr_tweet)

In [16]:
model.Count_unigram

{'lo': 644,
 'peor': 37,
 'de': 3357,
 'todo': 218,
 'es': 867,
 'que': 3383,
 'no': 1824,
 'me': 1613,
 'dan': 33,
 'por': 763,
 'un': 697,
 'tiempo': 42,
 'y': 2266,
 'luego': 50,
 '<unk>': 28023,
 'estoy': 241,
 'hasta': 199,
 'la': 2433,
 'verga': 1148,
 '</s>': 5544,
 'a': 2629,
 'seas': 23,
 'putos': 831,
 'minutos': 33,
 'después': 42,
 'en': 1191,
 '3': 63,
 'horas': 37,
 '?': 786,
 'más': 314,
 'unos': 45,
 'mi': 718,
 '!': 1505,
 'madre': 1079,
 'ese': 124,
 'joto': 209,
 'el': 1303,
 'marica': 183,
 'ex': 29,
 'tiene': 123,
 'así': 173,
 'uno': 58,
 'puede': 51,
 '"': 652,
 'su': 549,
 '😂': 377,
 'mujer': 55,
 'pinche': 296,
 'esta': 198,
 'se': 969,
 'loca': 556,
 '.': 2774,
 'tienen': 104,
 'como': 566,
 'mejor': 143,
 'ustedes': 47,
 'si': 662,
 'andar': 30,
 'pero': 439,
 'porque': 230,
 'mamar': 61,
 'hijos': 109,
 'chingada': 70,
 '@usuario': 1263,
 'te': 693,
 'digo': 54,
 'esa': 126,
 'está': 189,
 'buena': 39,
 'odio': 20,
 'los': 888,
 '😡': 92,
 'tu': 525,
 'siempr